### Analisis de incidencias delictivas en el estado de Sonora
Utilizando los datos proporcionados por el gobierno de México acerca de las incidencias delictivas en el país, se creo una base de datos filtrando la información para obtener las incidencias delictivas en el estado.

En la presente libreta se realizan las consultas necesarias para realizar un analisis de las denuncias en Sonora desde 2015 hasta 2021, finalizando con la ilustración de los datos obtenidos mediante una gráfica treemap.

#### Importamos las librerias que utilizaremos

In [1]:
import db_functions as mysql
import pandas as pd
from matplotlib import pyplot as plt
import os

Nos conectamos a la base de datos *denuncias*

In [2]:
# Loading credentials
if(os.environ.get("DB_TYPE") == None):
    from dotenv import load_dotenv
    from config.definitions import ROOT_DIR
    print(ROOT_DIR)
    load_dotenv(os.path.join(ROOT_DIR, 'config', 'conf', '.env'))

C:/Users/abrah/Documents/Repos/data-visualization


In [3]:
connection = mysql.create_db_connection("localhost",os.environ.get("DB_USER"),os.environ.get("DB_PASS"),os.environ.get("DB_NAME"))

MySQL DB connection succesful


Para el analisis debemos tener en cuenta los siguientes puntos:
 * Utilizaremos la vista **v_delitos_son** para realizar nuestras consultas a la base de datos
 * Necesitamos filtrar la información por año.
 * Nos interesa la frecuencia con la que ocurrio un tipo de delito.
 * Realizaremos un treemap por año y uno final que contenga todos.

In [4]:
# Tipos de delito
query = """
SELECT DISTINCT TipoDelito FROM v_delitos_son;
"""

In [5]:
tipos = mysql.read_query(connection,query)

In [6]:
tipos_l = [t[0] for t in tipos]

In [7]:
tipos_l

['Lesiones',
 'Tráfico de menores',
 'Otros delitos que atentan contra la libertad personal',
 'Corrupción de menores',
 'Homicidio',
 'Rapto',
 'Trata de personas',
 'Feminicidio',
 'Otros delitos que atentan contra la vida y la integridad corporal',
 'Secuestro',
 'Otros delitos contra la sociedad',
 'Extorsión',
 'Aborto']

### Incidencias delictivas en Sonora en 1995

In [14]:
### Incidencias delictivas en Sonora en 1995
# Query
query = """
SELECT TipoDelito, SUM(Ocurrencias) FROM v_delitos_son WHERE YEAR(Fecha) = 2015 GROUP BY TipoDelito;
"""

In [15]:
del_1995 = mysql.read_query(connection,query)

In [16]:
print(del_1995)

[('Lesiones', Decimal('3858')), ('Tráfico de menores', Decimal('44')), ('Otros delitos que atentan contra la libertad personal', Decimal('211')), ('Corrupción de menores', Decimal('139')), ('Homicidio', Decimal('1135')), ('Feminicidio', Decimal('24')), ('Rapto', Decimal('21')), ('Extorsión', Decimal('23')), ('Otros delitos que atentan contra la vida y la integridad corporal', Decimal('3')), ('Secuestro', Decimal('10')), ('Aborto', Decimal('3'))]


In [17]:
connection.close()